# Landsat 8: 
## EDS 220, Fall 2021
Friday, November 19

## Authors

- Julia Parish, UC Santa Barbara (jparish@bren.ucsb.edu)
- Alex Vand, UC Santa Barbara (asy@bren.ucsb.edu)
- Wylie Hampson, UC Santa Barbara (whampson@bren.ucsb.edu)
- Shale Hunter, UC Santa Barbara (shale@bren.ucsb.edu)

## Table of Contents

[1. Purpose](#purpose)

[2. Dataset Description](#overview)

[3. Data I/O](#io)

[4. Metadata Display and Basic Visualization](#display)

[5. Use Case Examples](#usecases)

[6. Create Binder Environment](#binder)

[7. References](#references)

<a id='purpose'></a> 
### Notebook Purpose

This notebook was created to provide an introduction to using satellite data from the <a href="https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2#description/" target="_blank">USGS Landsat 8 Level 2, Collection 2, Tier 1</a> sourced from Google Earth Engine for analysis to identify soil composition.


<a id='overview'></a> 
### Dataset Description

The dataset JAWS is exploring is <a href="https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2#description/" target="_blank">USGS Landsat 8 Level 2, Collection 2, Tier 1</a> sourced from Google Earth Engine. The creators of the dataset is a Joint program of USGS and NASA. Landsat8 is a joint effort between USGS and NASA, with data collected from 2013-04-11 to the present. This particular dataset is the atmospherically corrected surface reflectance and land surface temperature from the Landsat 8 OLI/TIRS sensors. The satellite orbits the earth in 99 minutes, covering individual overlapping swaths, or “scenes”, of the surface. Images are collected over 8-16 days and then compiled to create comprehensive images for the entire earth. Landsat data is represented in WGS84 and the images contain 5 visible and near-infrared bands, 2 short-wave infrared bands, and 1 thermal infrared.

Images are provided as TIFs, which can support the wide variety of image types already used for geographic imagery. Metadata files are available for download in Material Library File (MTL) and Extensible Markup Language (XML) formats.


- The source/archive you will be using to retrieve the data: Google Earth Engine <a href="https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2#description/" target="_blank">USGS Landsat 8 Level 2, Collection 2, Tier 1</a>

- Any known issues with data quality that might be expected to impact the results<br>
Since the data is coming from Landsat 8, we can trust that it is generally pretty reliable data. The remote sensing equipment on Landsat 8 is the Operational Land Imager (OLI) which is a multichannel scanning radiometer,and a two-channel infared radiometer, Thermal Infrared Sensor (TIRS).<br>
There are some known issues with this dataset, but many of them don’t concern most people that may be using this data. One of the most concerning issues is with cloud cover and cloud shadows. The data from this dataset can be affected if there is heavy cloud coverage. The images from the dataset have a 30 meter resolution which will allow us to get high definition images for the area of land that we will be looking at. If you were to look at a small area of land, say a few square kilometers, this data set may not be ideal because the 30 meter resolution won’t allow you to see much detail, but for this use case it works well.

<a id='io'></a> 
### Dataset Input/Output 

Setup for our analysis is quite simple using the Landsat API:

1) Import necessary packages

In [1]:
import ee
import geemap
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
ee.Authenticate()
ee.Initialize()

2) Set parameters that will be needed for analysis in subsequent portions of the notebook.

Here, we create a region of interest with a 50km radius around the coordinates (32.05, -104.74). 

In [4]:
# create a centroid over region of interest, Guadalupe Mountains of New Mexico and Texas

center_lat = 32.05
center_lon = -104.74
center_point = ee.Geometry.Point(center_lon, center_lat)
roi = center_point.buffer(50000)

3) Read in the data from the Landsat API.
    Note the last part of the identifier indicates that we are using Tier 1 data from NASA's Collection 2 of Landsat data.

In [5]:
# read in Landsat 8 data from Google Earth Engine

data = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")

<a id='display'></a> 
### Metadata Display and Basic Visualization

To get an idea of what we're working with, we can use the `propertyNames()` and `getInfo()` function to display some metadata for Landsat 8. For our purposes, available bands are the most important. A reminder of what wavelengths are recorded in which band can be found [here](https://www.usgs.gov/media/images/landsat-8-band-designations).

An important thing to keep track of with Landsat data is the dates: As satellites cycle through (i.e. break and are replaced), Landsat data must be accessed through different APIs. Landsat 8 data is only availble from February 2013, so if we were interested in images from before then we would have to access Landsat 7 data, whihc runs back to April 1999, and so on.

In [8]:
data.first().propertyNames()

In [48]:
# Explore the metadata - metadata display: variables
testImg = data.first()
properties = testImg.propertyNames()
print('Metadata properties:',
      properties.getInfo())  # ee.List of metadata properties


Metadata properties: ['DATA_SOURCE_ELEVATION', 'WRS_TYPE', 'system:id', 'REFLECTANCE_ADD_BAND_1', 'REFLECTANCE_ADD_BAND_2', 'DATUM', 'REFLECTANCE_ADD_BAND_3', 'REFLECTANCE_ADD_BAND_4', 'REFLECTANCE_ADD_BAND_5', 'REFLECTANCE_ADD_BAND_6', 'REFLECTANCE_ADD_BAND_7', 'system:footprint', 'REFLECTIVE_SAMPLES', 'system:version', 'GROUND_CONTROL_POINTS_VERSION', 'SUN_AZIMUTH', 'DATA_SOURCE_TIRS_STRAY_LIGHT_CORRECTION', 'UTM_ZONE', 'DATE_ACQUIRED', 'ELLIPSOID', 'system:time_end', 'DATA_SOURCE_PRESSURE', 'LANDSAT_PRODUCT_ID', 'STATION_ID', 'TEMPERATURE_ADD_BAND_ST_B10', 'DATA_SOURCE_REANALYSIS', 'REFLECTANCE_MULT_BAND_7', 'system:time_start', 'REFLECTANCE_MULT_BAND_6', 'L1_PROCESSING_LEVEL', 'PROCESSING_SOFTWARE_VERSION', 'L1_DATE_PRODUCT_GENERATED', 'ORIENTATION', 'REFLECTANCE_MULT_BAND_1', 'WRS_ROW', 'REFLECTANCE_MULT_BAND_3', 'REFLECTANCE_MULT_BAND_2', 'TARGET_WRS_ROW', 'REFLECTANCE_MULT_BAND_5', 'REFLECTANCE_MULT_BAND_4', 'THERMAL_LINES', 'TIRS_SSM_POSITION_STATUS', 'GRID_CELL_SIZE_THERMAL', 

In [49]:
# Explore the metadata - metadata display: coordinate information, parameters, information on missing data

bands = testImg.bandNames()
str(bands.getInfo())
testImg.getInfo()

{'type': 'Image',
 'bands': [{'id': 'SR_B1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [9161, 9161],
   'crs': 'EPSG:32628',
   'crs_transform': [30, 0, 341085, 0, -30, 8808015]},
  {'id': 'SR_B2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [9161, 9161],
   'crs': 'EPSG:32628',
   'crs_transform': [30, 0, 341085, 0, -30, 8808015]},
  {'id': 'SR_B3',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [9161, 9161],
   'crs': 'EPSG:32628',
   'crs_transform': [30, 0, 341085, 0, -30, 8808015]},
  {'id': 'SR_B4',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [9161, 9161],
   'crs': 'EPSG:32628',
   'crs_transform': [30, 0, 341085, 0, -30, 8808015]},
  {'id': 'SR_B5',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
 

In [50]:
# Filter data for mean values from September 2021
#data = data.filterBounds(roi).filter(ee.Filter.date('2021-09-01', '2021-09-07')) # Image collection


In [79]:
# cloud filter

data = data.filter('CLOUD_COVER < 20')

In [80]:


data_mean = data.filterBounds(roi).filter(ee.Filter.date('2021-07-01', '2021-10-30')).mean() # Single image

In [81]:
# The bands that we are interested in. 
# SR_B1 (band 1) is useful for coastal and aerosol studies. 
# SR_B5 (band 5) emphasizes biomass content and shorelines.
# SR_B7 (band 7) Improved moisture content of soil and vegetation; penetrates thin clouds.
visParams = {'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
             'min': 1,
             'max': 65455
            }

In [82]:
# Basic map of with data from the bands that we are looking at.
map = geemap.Map(center = [center_lat, center_lon], zoom = 7)
map.addLayer(data_mean, visParams)
map

Map(center=[32.05, -104.74], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(child…

<a id='usecase'></a> 
### Use Case Example: Identifying Gypsic Soils

#### Mapping habitat suitability for the identification and conservation of the Guadalupe Mountain necklacepod (*Sophora gypsophila*)

You don't have to be a rocket scientist to use satellite data! This part of the tutorial will focus on an application useful to many conservation ecologists: identifying potential habitat for a threatened species.

_Sophora gypsophila_ is a G1 Critically Endangered shrub in the pea family endemic to a small area surrounding the Guadalupe mountains in Southeastern New Mexico and Western Texas. Part of what puts _S. gypsophila_ at suck risk is the fact that it is a substrate obligate - in other words, it can only survive in a specific soil type. But this also gives us an advantage: it means that we can identify potentially suitable habitat simply by mapping the gypsic areas in the vicinity of existing populations of _S. gypsophila_ and subsetting these areas by the plant's known elevation range. This can be done using Landsat bands 1, 5, and 7, taking advantage of the fact that gypsic soils are highly reflective in band 5 () but particularly nonreflective in band 7 ().

An additional step in adapting Landsat data for conservation purposes would be comparing our identified habitat with land ownership maps in the region to target separate management solutions for public and private land, but we will not explore that step in this notebook.

In [59]:
nir = data_mean.select('SR_B5') # Near infrared - Emphasizes biomass content and shorelines.
swir2 = data_mean.select('SR_B7') # Short wave infrared - Improved moisture content of soil and vegetation.

oif = (nir.subtract(swir2)).divide((nir.add(swir2))).rename('OIF') # optimum index factor statistical value to select the optimum combination of three bands for a satellite image

data_oif = data_mean.addBands(oif)

Now we can redefine the band parameters we are mapping to look at OIF specifically:

In [62]:
visParamsoif = {'bands': ['OIF'],
             'min': -1,
             'max': 1,
             'palette': ['white', 'green']
            }

In [63]:
map_oif = geemap.Map(center = [center_lat, center_lon], zoom = 7)
legends = geemap.builtin_legends
for legend in legends :
    print(legend)
map_oif.addLayer(data_oif, visParamsoif)
#map_oif.add_legend(builtin_legend = "NLCD") # Experimenting with legends
map_oif

NLCD
NWI
MODIS/051/MCD12Q1
GLOBCOVER
JAXA/PALSAR
MODIS/006/MCD12Q1
Oxford
AAFC/ACI
COPERNICUS/CORINE/V20/100m
COPERNICUS/Landcover/100m/Proba-V/Global
USDA/NASS/CDL


Map(center=[32.05, -104.74], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(child…

In [13]:
data_oif_mask = data_oif.mask('need conditional here')

<a id='binder'></a> 
### Create Binder Environment

<a id='references'></a> 
### References

Binte Mostafiz, R.; Noguchi, R.; Ahamed, T. Agricultural Land Suitability Assessment Using Satellite Remote Sensing-Derived Soil-Vegetation Indices. Land 2021, 10, 223. https://doi.org/10.3390/land10020223

Boettinger, J., Ramsey, R., Bodily, J., Cole, N., Kienast-brown, S., Nield, S., Saunders, A., & Stum, A. (2008). Landsat Spectral Data for Digital Soil Mapping. Digital Soil Mapping with Limited Data. https://doi.org/10.1007/978-1-4020-8592-5_16

Browning, D. M., & Duniway, M. C. (2011). Digital Soil Mapping in the Absence of Field Training Data: A Case Study Using Terrain Attributes and Semiautomated Soil Signature Derivation to Distinguish Ecological Potential. Applied and Environmental Soil Science, 2011, e421904. https://doi.org/10.1155/2011/421904

Landsat 8 data courtesy of the U.S. Geological Survey. https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2#image-properties

Nield, S. J., Boettinger, J. L., & Ramsey, R. D. (2007). Digitally Mapping Gypsic and Natric Soil Areas Using Landsat ETM Data. Soil Science Society of America Journal, 71(1), 245–252. https://doi.org/10.2136/sssaj2006-0049

Ridwan, M. A., Radzi, N., Ahmad, W., Mustafa, I., Din, N., Jalil, Y., Isa, A., Othman A., Zaki, W. Applications of Landsat-8 Data: Survey. Internation Journal of Engineering & Techonology 2018, 11, 436-441. DOI:10.14419/ijet.v7i4.35.22858

Sophora gypsophila | NatureServe Explorer. (n.d.). Retrieved November 16, 2021, from https://explorer.natureserve.org/Taxon/ELEMENT_GLOBAL.2.155365/Sophora_gypsophila

Soil Survey Manual - Ch. 5. Digital Soil Mapping | NRCS Soils. (n.d.). Retrieved November 16, 2021, from https://www.nrcs.usda.gov/wps/portal/nrcs/detail/soils/ref/?cid=nrcs142p2_054255

Texas Native Plants Database. (n.d.). Retrieved November 16, 2021, from https://aggie-horticulture.tamu.edu/ornamentals/nativeshrubs/sophoragypsophila.htm

Vermote, E., Justice, C., Claverie, M., & Franch, B. (2016). Preliminary analysis of the performance of the Landsat 8/OLI land surface reflectance product. Remote Sensing of Environment, 185, 46-56.
